# Homework 1
## Anna Tigranyan

#### Simple RecSys

##### Using movies.csv and ratings.csv create Simple Recommender.

Restrictions
   - Use only pandas package
   - You can use any other packages only for visualisations

--------------------------------------

We will use **weighted rating** formula as a metric.

$WR = (V / (V + M)) * R + (M / (V + M)) * C$

Where:

- V is the number of views for the movie
- M is the minimum views requried to be listed in the chart
- R is the average rating of the movie
- C is the mean view across the whole report

In [1]:
import pandas as pd

In [2]:
# importing datasets

mov = pd.read_csv("movies.csv")
rat = pd.read_csv("ratings.csv")

In [3]:
mov.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rat.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Let's calculate the number of views for each film by **groupbying** rat dataset (**'userId'**) by **'movieId'** and counting.

In [5]:
n_views = rat[['userId','movieId']].groupby('movieId').count()


Now let's calculate the average rating for each film by **groupbying** rat dataset (**'rating'**) by **'movieId'** and calculating the mean .

In [6]:
avg_rat = rat[['rating','movieId']].groupby('movieId').mean()

Finally to have our final dataset **df** let's **join** **n_views** and **avg_rat** columns to our mov dataset by **'movieId'**

In [7]:
df = pd.merge(n_views, mov, on='movieId', how='inner')
df = pd.merge(avg_rat, df, on='movieId', how='inner')

In [8]:
df.head()

,movieId,rating,userId,title,genres
0,1,3.920930,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.431818,110,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.259615,52,Grumpier Old Men (1995),Comedy|Romance
3,4,2.357143,7,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.071429,49,Father of the Bride Part II (1995),Comedy


--------------------------------------

**V** - the number of views for the movie:

In [9]:
V = df['userId']
V.head()

0    215
1    110
2     52
3      7
4     49
Name: userId, dtype: int64

**M** - the minimum views requried to be listed in the chart

In [10]:
M = df['userId'].quantile(0.90)
M

27.0

**R** - the average rating of the movie

In [11]:
R = df['rating']
R.head()

0    3.920930
1    3.431818
2    3.259615
3    2.357143
4    3.071429
Name: rating, dtype: float64

**C** - the mean view across the whole report

In [12]:
C = rat['rating'].mean() 
C

3.501556983616962

Let's write function computing the weighted rating of each movie based on the IMDB formula:

In [13]:
def weighted_rating(x, M=M, C=C):
    V = x['userId']
    R = x['rating']
    return (V / (V + M) * R) + (M / (M + V) * C)

Before defining a new feature 'score' we need to filter our df in order to have qualified movies( movies that have M and more views ):

In [14]:
q_df = df.copy().loc[df['userId'] >= M]

In [15]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_df['score'] = q_df.apply(weighted_rating, axis=1)
q_df.head()

,movieId,rating,userId,title,genres,score
0,1,3.920930,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.874141
1,2,3.431818,110,Jumanji (1995),Adventure|Children|Fantasy,3.445562
2,3,3.259615,52,Grumpier Old Men (1995),Comedy|Romance,3.342304
4,5,3.071429,49,Father of the Bride Part II (1995),Comedy,3.224237
5,6,3.946078,102,Heat (1995),Action|Crime|Thriller,3.853039


Let's sort movies baced on above score calculation: 

In [16]:
q_df = q_df.sort_values('score', ascending=False)

In [17]:
# Printing the top 10 movies
q_df[['title', 'genres', 'rating', 'score']].head(10)

,title,genres,rating,score
277,"Shawshank Redemption, The (1994)",Crime|Drama,4.429022,4.356227
659,"Godfather, The (1972)",Crime|Drama,4.289062,4.191973
2224,Fight Club (1999),Action|Crime|Drama|Thriller,4.272936,4.187927
224,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.231076,4.160223
46,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.237745,4.151697
461,Schindler's List (1993),Drama|War,4.225000,4.145919
257,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.197068,4.140844
897,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,4.215640,4.134630
1938,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.192446,4.131285
921,"Godfather: Part II, The (1974)",Crime|Drama,4.259690,4.128475
